<a href="https://colab.research.google.com/github/amphyxs/know-graphs-cw/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -r requirements.txt 

Defaulting to user installation because normal site-packages is not writeable


In [25]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer
from typing import Optional, List
import re
import logging

# Настройка логгирования

In [26]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Инициализиация онтологии

In [27]:
g = Graph()

g.parse('ontology.rdf')
ONTOLOGY_IRI = 'http://www.semanticweb.org/user/ontologies/2024/9/music-knowledge'

def ref(resource: str) -> URIRef:
  return URIRef(f'{ONTOLOGY_IRI}#{resource}')

# Python-объекты для классов в онтологии

In [28]:
class OntologyClass:
  class_name: str
  _class_uri_ref: Optional[URIRef] = None
  uri_ref: URIRef

  @property
  def class_uri_ref(self) -> URIRef:
    if not self._class_uri_ref:
      self._class_uri_ref = ref(self.class_name)

    return self._class_uri_ref

  def __init__(self, name: str, **kwargs: dict) -> None:
    self.uri_ref = ref(self._prepare_name_for_ref(name))
    g.add((self.uri_ref, RDF.type, self.class_uri_ref))

  def _prepare_name_for_ref(self, name: str) -> str:
    name = name.replace(' ', '_')

    return re.sub(r'[^A-Za-z0-9_]', '', name)

In [29]:
class Artist(OntologyClass):
  class_name = 'Исполнитель'

class Song(OntologyClass):
  owns_object_property = ref('владеть')
  class_name = 'Композиция'

  def __init__(self, name: str, owner: Artist, **kwargs: dict) -> None:
    super().__init__(name, **kwargs)
    g.add((owner.uri_ref, self.owns_object_property, self.uri_ref))

class Album(OntologyClass):
    class_name = 'Альбом'

    def __init__(self, name: str, **kwargs: dict) -> None:
        super().__init__(name, **kwargs)

class Country(OntologyClass):
    class_name = 'Страна'

    def __init__(self, name: str, **kwargs: dict) -> None:
        super().__init__(name, **kwargs)


# Словарь для определения стран

In [30]:
COUNTRY_CODES = {
    "USA": "США",
    "GB": "Великобритания",
    "CA": "Канада",
    "DE": "Германия",
    "FR": "Франция",
    "RU": "Россия",
    "AU": "Австралия",
    "JP": "Япония",
    "BR": "Бразилия",
    "IN": "Индия",
}

# Свойства

In [31]:
belongs_to_country_property = ref('принадлежит_стране')

belongs_to_album_property = ref('принадлежит_альбому')

# Получение инстансов для классов из API

In [34]:
import requests

API_URL = "https://itunes.apple.com/search"

def fetch_musical_bands(limit: int = 10) -> List[str]:
    url = API_URL
    params = {
        "term": "band",        # General search term to find bands
        "media": "music",
        "entity": "musicArtist",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [artist['artistName'] for artist in results]

def fetch_songs_for_band(band_name: str, limit: int = 10) -> List[str]:
    url = API_URL
    params = {
        "term": band_name,
        "media": "music",
        "entity": "song",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [song['trackName'] for song in results]

def fetch_album_for_song(song_name: str, artist_name: str) -> str:
    url = API_URL
    params = {
        "term": f"{song_name} {artist_name}",
        "media": "music",
        "entity": "album",
        "limit": 1,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    if results:
        return results[0]['collectionName']
    return "Unknown Album"

def fetch_country_for_song(song_name: str, artist_name: str) -> str:
    url = API_URL
    params = {
        "term": f"{song_name} {artist_name}",
        "media": "music",
        "entity": "song",
        "limit": 1,
    }

    # logging.info(f'Запрос к iTunes API для песни: {song_name} исполнителя: {artist_name}')

    response = requests.get(url, params=params)
    # logging.info(f'Ответ от iTunes API: {response.json()}')

    results = response.json().get("results", [])
    
    if results:
        country_code = results[0].get('country')  # Получаем код страны
        if country_code:
        #     logging.info(f'Код страны для песни "{song_name}" исполнителя "{artist_name}": {country_code}')
            return COUNTRY_CODES.get(country_code, "Неизвестная страна")  # Возвращаем название страны или "Неизвестная страна"
        else:
        #     logging.warning(f'Код страны отсутствует для песни "{song_name}" исполнителя: {artist_name}')
            return "Неизвестная страна"
    # logging.warning(f'Страна не найдена для песни "{song_name}" исполнителя: {artist_name}')
    return "Неизвестная страна"

bands = fetch_musical_bands()
for band in bands:
    songs = fetch_songs_for_band(band)
    artist_instance = Artist(band)

    for song in songs:
        album_name = fetch_album_for_song(song, band)  # Получаем альбом для песни
        album_instance = Album(album_name)  # Создаем экземпляр альбома
        g.add((album_instance.uri_ref, RDF.type, album_instance.class_uri_ref))  # Добавляем альбом в граф

        # Устанавливаем связь между песней и альбомом
        song_instance = Song(song, artist_instance)
        g.add((song_instance.uri_ref, belongs_to_album_property, album_instance.uri_ref))  # Устанавливаем связь между песней и альбомом

In [35]:
g.serialize(destination='ontology-from-py.rdf')

<Graph identifier=N7726bfa40be041f09bf3b897d6059671 (<class 'rdflib.graph.Graph'>)>